# Advanced Slicing Techniques

**Module 05 | Notebook 03**

---

## Objective
By the end of this notebook, you will master:
- Views vs copies in depth
- Stride tricks and memory layout
- Structured and record arrays
- Advanced slicing patterns
- Index arrays for complex operations

In [3]:
import numpy as np
np.set_printoptions(precision=2)

---
## 1. Views vs Copies: Deep Dive

In [4]:
# Operations that return VIEWS (no copy)
arr = np.arange(12).reshape(3, 4)
print(f"Original:\n{arr}")

# Basic slicing
view1 = arr[1:3]
print(f"Slice is view: {np.shares_memory(arr, view1)}")

# Reshape (usually)
view2 = arr.reshape(4, 3)
print(f"Reshape is view: {np.shares_memory(arr, view2)}")

# Transpose
view3 = arr.T
print(f"Transpose is view: {np.shares_memory(arr, view3)}")

# Ravel (when possible)
view4 = arr.ravel()
print(f"Ravel is view: {np.shares_memory(arr, view4)}")

Original:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Slice is view: True
Reshape is view: True
Transpose is view: True
Ravel is view: True


In [5]:
# Operations that return COPIES
arr = np.arange(12).reshape(3, 4)

# Fancy indexing
copy1 = arr[[0, 2]]
print(f"Fancy index is view: {np.shares_memory(arr, copy1)}")

# Boolean indexing
copy2 = arr[arr > 5]
print(f"Boolean index is view: {np.shares_memory(arr, copy2)}")

# Flatten (always copy)
copy3 = arr.flatten()
print(f"Flatten is view: {np.shares_memory(arr, copy3)}")

# Explicit copy
copy4 = arr.copy()
print(f"Copy is view: {np.shares_memory(arr, copy4)}")

Fancy index is view: False
Boolean index is view: False
Flatten is view: False
Copy is view: False


In [6]:
# Check if array owns its data
arr = np.arange(10)
view = arr[2:8]

print(f"arr owns data: {arr.flags['OWNDATA']}")
print(f"view owns data: {view.flags['OWNDATA']}")

# Check base
print(f"arr.base: {arr.base}")
print(f"view.base is arr: {view.base is arr}")

arr owns data: True
view owns data: False
arr.base: None
view.base is arr: True


---
## 2. Memory Layout and Strides

In [7]:
# Strides: bytes to jump to next element in each dimension
arr = np.arange(12).reshape(3, 4)
print(f"Array:\n{arr}")
print(f"Shape: {arr.shape}")
print(f"Strides: {arr.strides}")
print(f"Item size: {arr.itemsize} bytes")

Array:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Shape: (3, 4)
Strides: (32, 8)
Item size: 8 bytes


In [8]:
# C-order (row-major) vs Fortran-order (column-major)
c_arr = np.arange(12).reshape(3, 4, order='C')
f_arr = np.arange(12).reshape(3, 4, order='F')

print(f"C-order strides: {c_arr.strides}")
print(f"F-order strides: {f_arr.strides}")

C-order strides: (32, 8)
F-order strides: (8, 24)


In [9]:
# Understanding stride behavior
arr = np.arange(20).reshape(4, 5)

# Stepping with slice
stepped = arr[::2, ::2]
print(f"Original strides: {arr.strides}")
print(f"Stepped strides: {stepped.strides}")
print(f"Stepped array:\n{stepped}")

Original strides: (40, 8)
Stepped strides: (80, 16)
Stepped array:
[[ 0  2  4]
 [10 12 14]]


In [10]:
# Transposing changes strides, not data
arr = np.arange(12).reshape(3, 4)
trans = arr.T

print(f"arr strides: {arr.strides}")
print(f"T strides: {trans.strides}")
print(f"Same memory: {np.shares_memory(arr, trans)}")

arr strides: (32, 8)
T strides: (8, 32)
Same memory: True


---
## 3. Stride Tricks

In [11]:
from numpy.lib.stride_tricks import as_strided, sliding_window_view

In [12]:
# Create sliding windows without copying
arr = np.arange(10)
windows = sliding_window_view(arr, window_shape=3)

print(f"Array: {arr}")
print(f"Windows:\n{windows}")
print(f"Shares memory: {np.shares_memory(arr, windows)}")

Array: [0 1 2 3 4 5 6 7 8 9]
Windows:
[[0 1 2]
 [1 2 3]
 [2 3 4]
 [3 4 5]
 [4 5 6]
 [5 6 7]
 [6 7 8]
 [7 8 9]]
Shares memory: True


In [13]:
# 2D sliding windows (image patches)
image = np.arange(16).reshape(4, 4)
patches = sliding_window_view(image, window_shape=(2, 2))

print(f"Image:\n{image}")
print(f"Patches shape: {patches.shape}")
print(f"Top-left patch:\n{patches[0, 0]}")

Image:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
Patches shape: (3, 3, 2, 2)
Top-left patch:
[[0 1]
 [4 5]]


In [14]:
# as_strided for custom views (USE WITH CAUTION!)
arr = np.arange(10)

# Create 2D view: repeat first 5 elements 3 times
# This is advanced and can cause crashes if misused!
view = as_strided(arr, shape=(3, 5), strides=(0, arr.strides[0]))
print(f"Repeated view (stride trick):\n{view}")

Repeated view (stride trick):
[[0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]]


---
## 4. Structured Arrays

In [15]:
# Define structured dtype
dt = np.dtype([('name', 'U10'), ('age', 'i4'), ('weight', 'f8')])

# Create structured array
data = np.array([('Alice', 25, 55.5), ('Bob', 30, 70.2), ('Charlie', 35, 80.1)], dtype=dt)
print(f"Structured array:\n{data}")
print(f"Dtype: {data.dtype}")

Structured array:
[('Alice', 25, 55.5) ('Bob', 30, 70.2) ('Charlie', 35, 80.1)]
Dtype: [('name', '<U10'), ('age', '<i4'), ('weight', '<f8')]


In [16]:
# Access fields
print(f"Names: {data['name']}")
print(f"Ages: {data['age']}")
print(f"Weights: {data['weight']}")

Names: ['Alice' 'Bob' 'Charlie']
Ages: [25 30 35]
Weights: [55.5 70.2 80.1]


In [17]:
# Filter by condition
adults = data[data['age'] >= 30]
print(f"Age >= 30:\n{adults}")

Age >= 30:
[('Bob', 30, 70.2) ('Charlie', 35, 80.1)]


In [18]:
# Sort by field
sorted_by_age = np.sort(data, order='age')
print(f"Sorted by age:\n{sorted_by_age}")

Sorted by age:
[('Alice', 25, 55.5) ('Bob', 30, 70.2) ('Charlie', 35, 80.1)]


In [19]:
# Access multiple fields
subset = data[['name', 'age']]
print(f"Name and age:\n{subset}")

Name and age:
[('Alice', 25) ('Bob', 30) ('Charlie', 35)]


---
## 5. Record Arrays

In [20]:
# Record arrays allow attribute access
rec = np.rec.array([('Alice', 25, 55.5), ('Bob', 30, 70.2)],
                   dtype=[('name', 'U10'), ('age', 'i4'), ('weight', 'f8')])

print(f"Record array:\n{rec}")

# Attribute access (instead of ['name'])
print(f"Names: {rec.name}")
print(f"Ages: {rec.age}")

Record array:
[('Alice', 25, 55.5) ('Bob', 30, 70.2)]
Names: ['Alice' 'Bob']
Ages: [25 30]


In [21]:
# Convert structured to recarray
data = np.array([('Alice', 25), ('Bob', 30)], dtype=[('name', 'U10'), ('age', 'i4')])
rec = data.view(np.recarray)

print(f"Access via attribute: {rec.name}")

Access via attribute: ['Alice' 'Bob']


---
## 6. Advanced Slicing Patterns

In [22]:
# Pattern: Extract diagonal blocks
arr = np.arange(36).reshape(6, 6)
print(f"Array:\n{arr}")

# 2x2 diagonal blocks
block_size = 2
blocks = [arr[i:i+block_size, i:i+block_size] for i in range(0, 6, block_size)]
print(f"Diagonal blocks:")
for i, b in enumerate(blocks):
    print(f"Block {i}:\n{b}")

Array:
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]
 [30 31 32 33 34 35]]
Diagonal blocks:
Block 0:
[[0 1]
 [6 7]]
Block 1:
[[14 15]
 [20 21]]
Block 2:
[[28 29]
 [34 35]]


In [23]:
# Pattern: Checkerboard selection
arr = np.arange(16).reshape(4, 4)
print(f"Array:\n{arr}")

# Select checkerboard pattern (like black squares on chess board)
checkerboard = arr[::2, ::2].flatten().tolist() + arr[1::2, 1::2].flatten().tolist()
print(f"Checkerboard elements: {checkerboard}")

Array:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
Checkerboard elements: [0, 2, 8, 10, 5, 7, 13, 15]


In [24]:
# Pattern: Reverse along specific axis
arr = np.arange(12).reshape(3, 4)
print(f"Original:\n{arr}")

print(f"Reverse rows:\n{arr[::-1, :]}")
print(f"Reverse cols:\n{arr[:, ::-1]}")
print(f"Reverse both:\n{arr[::-1, ::-1]}")

Original:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Reverse rows:
[[ 8  9 10 11]
 [ 4  5  6  7]
 [ 0  1  2  3]]
Reverse cols:
[[ 3  2  1  0]
 [ 7  6  5  4]
 [11 10  9  8]]
Reverse both:
[[11 10  9  8]
 [ 7  6  5  4]
 [ 3  2  1  0]]


In [25]:
# Pattern: Circular indexing (wrap-around)
arr = np.arange(10)
indices = np.array([8, 9, 10, 11, 12]) % len(arr)

print(f"Array: {arr}")
print(f"Wrapped indices: {indices}")
print(f"Circular access: {arr[indices]}")

Array: [0 1 2 3 4 5 6 7 8 9]
Wrapped indices: [8 9 0 1 2]
Circular access: [8 9 0 1 2]


In [26]:
# Pattern: Tile indices for batch operations
batch_size = 4
seq_len = 3

# Create indices for gathering from batch
batch_idx = np.arange(batch_size)[:, np.newaxis]  # (4, 1)
seq_idx = np.arange(seq_len)[np.newaxis, :]      # (1, 3)

print(f"Batch indices:\n{np.broadcast_arrays(batch_idx, seq_idx)[0]}")

Batch indices:
[[0 0 0]
 [1 1 1]
 [2 2 2]
 [3 3 3]]


---
## 7. Memory-Efficient Techniques

In [27]:
# Technique: Use views for large operations
large_arr = np.arange(1000000)

# This creates a view, not copy
first_half = large_arr[:500000]
print(f"View memory: {first_half.nbytes / 1e6:.1f} MB (but shares with original)")
print(f"Actual additional memory: 0 MB")

View memory: 4.0 MB (but shares with original)
Actual additional memory: 0 MB


In [28]:
# Technique: Modify in-place
arr = np.arange(10)

# Creates new array (uses memory)
# result = arr * 2

# In-place (no extra memory)
arr *= 2
print(f"In-place modified: {arr}")

In-place modified: [ 0  2  4  6  8 10 12 14 16 18]


In [29]:
# Technique: out parameter for ufuncs
arr = np.arange(10, dtype=float)
result = np.empty_like(arr)

# No temporary array created
np.sqrt(arr, out=result)
print(f"Result: {result}")

Result: [0.   1.   1.41 1.73 2.   2.24 2.45 2.65 2.83 3.  ]


In [30]:
# Technique: Use np.add.at for accumulation
arr = np.zeros(5)
indices = np.array([0, 1, 1, 2, 2, 2])
values = np.ones(6)

np.add.at(arr, indices, values)
print(f"Accumulated: {arr}")

Accumulated: [1. 2. 3. 0. 0.]


---
## Key Points Summary

**Views vs Copies:**
- Slicing, reshape, transpose -> VIEW
- Fancy/boolean indexing, flatten -> COPY
- Check with `np.shares_memory()` or `arr.flags['OWNDATA']`

**Strides:**
- Bytes to jump per dimension
- C-order: last dimension fastest
- F-order: first dimension fastest
- Stride of 0 = broadcasting trick

**Structured Arrays:**
- Named fields with different types
- Access: `arr['fieldname']`
- Record arrays: `arr.fieldname`

---
## Interview Tips

**Q1: How do you know if an operation returns a view or copy?**
> Use `np.shares_memory(a, b)` or check `arr.base`. Basic slicing returns views; fancy indexing, boolean indexing, and flatten() return copies.

**Q2: What are strides in NumPy?**
> Strides define how many bytes to skip in memory to move to the next element along each dimension. They enable views like transposes without copying data.

**Q3: When would you use structured arrays?**
> When you need tabular data with different types per column but want NumPy's speed benefits. Good for loading fixed-format binary files.

**Q4: How do you avoid memory issues with large arrays?**
> Use views instead of copies, in-place operations (`*=`, `+=`), `out` parameter in ufuncs, and process in chunks if needed.

---
## Practice Exercises

### Exercise 1: Identify view or copy

In [31]:
# Determine which operations create views vs copies
arr = np.arange(20).reshape(4, 5)

# Test these:
ops = [
    arr[1:3],
    arr[[1, 2]],
    arr.T,
    arr.flatten(),
    arr.ravel(),
    arr[arr > 10]
]


In [32]:
# Solution
arr = np.arange(20).reshape(4, 5)

print("Operation -> Shares memory (View/Copy)")
print(f"arr[1:3]: {np.shares_memory(arr, arr[1:3])} (View)")
print(f"arr[[1,2]]: {np.shares_memory(arr, arr[[1,2]])} (Copy)")
print(f"arr.T: {np.shares_memory(arr, arr.T)} (View)")
print(f"arr.flatten(): {np.shares_memory(arr, arr.flatten())} (Copy)")
print(f"arr.ravel(): {np.shares_memory(arr, arr.ravel())} (View)")
print(f"arr[arr>10]: {np.shares_memory(arr, arr[arr>10])} (Copy)")

Operation -> Shares memory (View/Copy)
arr[1:3]: True (View)
arr[[1,2]]: False (Copy)
arr.T: True (View)
arr.flatten(): False (Copy)
arr.ravel(): True (View)
arr[arr>10]: False (Copy)


### Exercise 2: Create structured array for student records

In [33]:
# Create structured array with: name (string), grade (int), gpa (float)
# Add 3 students, then filter by gpa > 3.5


In [34]:
# Solution
dt = np.dtype([('name', 'U20'), ('grade', 'i4'), ('gpa', 'f8')])
students = np.array([
    ('Alice', 12, 3.8),
    ('Bob', 11, 3.2),
    ('Charlie', 12, 3.9)
], dtype=dt)

print(f"All students:\n{students}")

honor_roll = students[students['gpa'] > 3.5]
print(f"Honor roll (GPA > 3.5):\n{honor_roll}")

All students:
[('Alice', 12, 3.8) ('Bob', 11, 3.2) ('Charlie', 12, 3.9)]
Honor roll (GPA > 3.5):
[('Alice', 12, 3.8) ('Charlie', 12, 3.9)]


### Exercise 3: Extract patches from image efficiently

In [35]:
# Extract all 3x3 patches from a 6x6 image using sliding_window_view
image = np.arange(36).reshape(6, 6)


In [36]:
# Solution
from numpy.lib.stride_tricks import sliding_window_view

image = np.arange(36).reshape(6, 6)
print(f"Image:\n{image}")

patches = sliding_window_view(image, window_shape=(3, 3))
print(f"Patches shape: {patches.shape}")
print(f"Number of patches: {patches.shape[0] * patches.shape[1]}")
print(f"First patch:\n{patches[0, 0]}")
print(f"Shares memory: {np.shares_memory(image, patches)}")

Image:
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]
 [30 31 32 33 34 35]]
Patches shape: (4, 4, 3, 3)
Number of patches: 16
First patch:
[[ 0  1  2]
 [ 6  7  8]
 [12 13 14]]
Shares memory: True


---
## Module 05 Complete!

You have mastered Advanced Indexing:
- Fancy Indexing
- Boolean Indexing
- Advanced Slicing Techniques

**Next Module:** 06_file_io - Saving, loading, and working with files!